In [2]:
from datasets import load_dataset
fever_wiki = load_dataset("fever", 'wiki_pages')
fever_nli = load_dataset("fever", 'v1.0')

/raid/nlp/sameer/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-25 00:27:06 - PyTorch version 2.2.1 available.
/raid/nlp/sameer/anaconda3/envs/myenv/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for fever contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fever
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/raid/nlp/sameer/anaconda3/envs/myenv/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for fever contains custom code which must be executed to c

In [3]:
corpus = fever_wiki['wikipedia_pages']["text"][1:]

In [3]:
import numpy as np

with open('/raid/nlp/sameer/fever/embeddings.npy', 'rb') as f:
    embeddings = np.load(f)

In [4]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Check if CUDA is available ans switch to GPU
if torch.cuda.is_available():
   model = model.to(torch.device("cuda:1"))
print(model.device)

queries = fever_nli["train"]["claim"]
query_embeddings = model.encode(queries, show_progress_bar=True)

cuda:1


Batches: 100%|██████████| 9733/9733 [01:36<00:00, 101.28it/s]


In [12]:
fever_wiki['wikipedia_pages']

Dataset({
    features: ['id', 'text', 'lines'],
    num_rows: 5416537
})

In [15]:
from tqdm import tqdm
wiki_id2lines = {}
for dump in tqdm(fever_wiki['wikipedia_pages']):
    if len(dump['text']) != 0:
        wiki_id2lines[dump['id']] = dump['text']

100%|██████████| 5416537/5416537 [02:30<00:00, 35879.68it/s]


In [16]:
cnt = 0
invalid_queries = []
query2evi = {}
for query in fever_nli["train"]:
    if query["evidence_sentence_id"] == -1:
        continue
    id_gold = query['evidence_wiki_url']
    try:
        query2evi[query["claim"]] = wiki_id2lines[id_gold]
    except:
        invalid_queries.append(query)

In [17]:
list(query2evi.items())[0]

('Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.',
 "Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰʌsd̥ɐ ˈʋald̥ɑʊ̯ -RSB- ; born 27 July 1970 -RRB- is a Danish actor , producer and screenwriter . He graduated from Danish National School of Theatre in Copenhagen in 1993 . Coster-Waldau 's breakthrough performance in Denmark was his role in the film Nightwatch -LRB- 1994 -RRB- . Since then he has appeared in numerous films in his native Scandinavia and Europe in general , including Headhunters -LRB- 2011 -RRB- and A Thousand Times Good Night -LRB- 2013 -RRB- .   In the United States , his debut film role was in the war film Black Hawk Down -LRB- 2001 -RRB- , playing Medal of Honor recipient Gary Gordon . He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam -LRB- 2008 -RRB- , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot . He became widely known to a broad audience fo

In [9]:
import faiss

index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(x=embeddings[1:], ids=np.array([i for i in range(len(corpus))]))

In [20]:
D, I = index.search(query_embeddings[:5000], k=10)

In [22]:
cnt  = 0
invalid_cnt = 0
for i in range(0,5000):
    # D,I = index.search(np.array([query_embeddings[i]]), k=10)
    try:
        if query2evi[queries[i]] in [corpus[m] for m in I[i]]:
            cnt += 1
    except:
        invalid_cnt += 1
print(cnt/(5000-invalid_cnt))

0.12833061446438282


In [13]:
print(query2evi[queries[0]])
print(queries[0])

Nikolaj Coster-Waldau -LRB- -LSB- neɡ̊olaɪ̯ kʰʌsd̥ɐ ˈʋald̥ɑʊ̯ -RSB- ; born 27 July 1970 -RRB- is a Danish actor , producer and screenwriter . He graduated from Danish National School of Theatre in Copenhagen in 1993 . Coster-Waldau 's breakthrough performance in Denmark was his role in the film Nightwatch -LRB- 1994 -RRB- . Since then he has appeared in numerous films in his native Scandinavia and Europe in general , including Headhunters -LRB- 2011 -RRB- and A Thousand Times Good Night -LRB- 2013 -RRB- .   In the United States , his debut film role was in the war film Black Hawk Down -LRB- 2001 -RRB- , playing Medal of Honor recipient Gary Gordon . He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam -LRB- 2008 -RRB- , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot . He became widely known to a broad audience for his current role as Ser Jaime Lannister , in the HBO series Game of 

In [18]:
[corpus[i] for i in I[0]]

['Panik may refer to :   Panik , one quarter of the production group Molemen .  Panik -LRB- band -RRB-  Panik , Armenia  Panik -LRB- film -RRB- , a 1928 German film directed by and starring Harry Piel  Richard Pánik , Slovak ice hockey player ',
 'Kahr may refer to :   Andrew Kahr , executive  Gustav Ritter von Kahr -LRB- 1862 -- 1934 -RRB- , politician  KAHR , radio station  Kahr Arms , manufacturer ',
 "Wildstein may refer to :  Skalná , a city in the Czech Republic known as `` Wildstein '' in German  Bronisław Wildstein -LRB- born 1952 -RRB- , a former Polish dissident , a journalist , and a freelance author  Wildstein list , a list distributed by Bronisław Wildstein of people associated with the Polish intelligence services  David Wildstein -LRB- born 1961 -RRB- , an American businessman , Republican Party politician , and political blogger  Dawid Wildstein , a Polish journalist ",
 "Derby is a 1949 West German sports film directed by Roger von Norman and starring Hannelore Schroth

# BM25 retriever

In [15]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [16]:
tokenized_query = fever_nli["train"][0]["claim"].split(" ")
retr_doc = bm25.get_top_n(tokenized_query, corpus, n=3)
print(retr_doc)

['A Second Chance -LRB- En chance til -RRB- is a 2014 Danish thriller film directed by Susanne Bier . The film stars Nikolaj Coster-Waldau , Ulrich Thomsen , Maria Bonnevie , Nikolaj Lie Kaas and Lykke May Andersen . It was screened in the Special Presentations section of the 2014 Toronto International Film Festival . ', 'Nikolaj is a given name -LRB- in most cases Николай , Slovene form of Nicholas transliterated to Danish or other Scandinavian language -RRB- , and may refer to :   Nikolaj Abraham Abildgaard , Danish artist  Nicolaj Agger , Danish professional football player  Nikolaj Coster-Waldau , Danish actor  Nikolaj Frederik Severin Grundtvig , Danish teacher , writer , poet , philosopher , historian , pastor and politician  Nikolaj Gavric , footballer from Switzerland who currently plays as midfielder for FC Concordia Basel in the Swiss Challenge League  Nikolaj Koppel , Danish musician  Nikolaj Hansen , Danish football player  Nikolaj Hübbe . balletmaster of the Royal Danish B

In [1]:
!pip install easy-elasticsearch

  Using cached elasticsearch-8.12.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached elastic_transport-8.12.0-py3-none-any.whl.metadata (3.5 kB)
Using cached elasticsearch-8.12.1-py3-none-any.whl (432 kB)
Using cached elastic_transport-8.12.0-py3-none-any.whl (59 kB)


In [7]:
from easy_elasticsearch import ElasticSearchBM25

# pool = {
#     'id1': 'What is Python? Is it a programming language',
#     'id2': 'Which Python version is the best?',
#     'id3': 'Using easy-elasticsearch in Python is really convenient!'
# }
pool = {i: corpus[i] for i in range(len(corpus))}
bm25 = ElasticSearchBM25(pool, port_http='9222', port_tcp='9333')  # By default, when `host=None` and `mode="docker"`, a ES docker container will be started at localhost.

2024-03-25 00:28:35 - Successfully reached out to ES service at http://localhost:9222
2024-03-25 00:28:35 - Successfully built connection to ES service at http://localhost:9222
2024-03-25 00:28:35 - No index found and now do indexing
100%|██████████| 10834/10834 [09:36<00:00, 18.78it/s]
2024-03-25 00:38:13 - Indexing work done: 5416536 documents indexed
2024-03-25 00:38:13 - All set up.


In [29]:
queries = fever_nli["train"]["claim"]

In [34]:
len(fever_nli["train"])

311431

In [ ]:
query2evi

In [21]:
query = queries[0]
rank = bm25.query(query, topk=10)  # topk should be <= 10000
# scores = bm25.score(query, document_ids=['id2', 'id3'])

print(query, rank)

Nikolaj Coster-Waldau worked with the Fox Broadcasting Company. {'5169564': "Ved verdens ende -LRB- At World 's End -RRB- is a 2009 Danish action comedy film directed by Tomas Villum Jensen and starring Nikolaj Lie Kaas , Birgitte Hjort Sørensen and Nikolaj Coster-Waldau . ", '3586361': 'Nukaaka Coster-Waldau -LRB- née Sascha Nukaka Motzfeldt ; born 23 February 1971 -RRB- , Nukâka , is a Greenlandic singer , actress , and a former Miss Greenland . She is married to Danish actor Nikolaj Coster-Waldau and has two daughters with him .   The Greenlandic actress of Inuit , German and Norwegian extraction was born in Uummannaq , Greenland . She is the daughter of Vivi and Josef Motzfeldt , a Greenlandic politician . ', '297368': 'A Second Chance -LRB- En chance til -RRB- is a 2014 Danish thriller film directed by Susanne Bier . The film stars Nikolaj Coster-Waldau , Ulrich Thomsen , Maria Bonnevie , Nikolaj Lie Kaas and Lykke May Andersen . It was screened in the Special Presentations sectio

In [36]:
cnt = 0
invalid_cnt = 0
for i in tqdm(range(0,10000)):
    rank = bm25.query(queries[i], topk=10)
    try:
        if query2evi[queries[i]] in rank.values():
            cnt += 1
    except:
        invalid_cnt += 1
print(cnt/(10000-invalid_cnt))

100%|██████████| 10000/10000 [04:05<00:00, 40.79it/s]

0.35321489001692047


In [6]:
bm25.delete_index()  # delete the one-trial index named 'one_trial'
bm25.delete_container()  # remove the docker container'

2024-03-25 00:28:21 - Index "one_trial" does not exist!
2024-03-25 00:28:21 - No running ES container found!
2024-03-25 00:28:21 - Found idling nodes:
 5bce22ba76ef   docker.elastic.co/elasticsearch/elasticsearch:7.9.1   "/tini -- /usr/local…"   About an hour ago   Up About an hour   0.0.0.0:9222->9200/tcp, :::9222->9200/tcp, 0.0.0.0:9333->9300/tcp, :::9333->9300/tcp   easy-elasticsearch-node1711301683
.
